In [ ]:
from Python.tigre.utilities.io.varian_cbct.xim_io import XIM
import os
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

In [ ]:
mydir = "C:/CBCT_Scatter_Removal_Project/data/clinical/4100758/2021-07-06_101037/11528a87-b894-4239-846d-45d8fd1efeda"

In [ ]:
folder = os.path.join(mydir, "Acquisitions")

# get filepaths
ximfilelist = []
for path in os.listdir(folder):
    if os.path.isdir(os.path.join(folder, path)):
        subdir = os.path.join(folder, path)
        for file in os.listdir(subdir):
             if file.startswith("Proj_") and file.endswith(".xim"):
                 ximfilelist.append(os.path.join(subdir,file))

In [ ]:
projections = []
angles = []
airnorm = []
for i,filepath in enumerate(tqdm(ximfilelist[0:1])):
    xim_img=XIM(filepath)
    airnorm.append(xim_img.properties["KVNormChamber"])
    angles.append(xim_img.properties["GantryRtn"])
    try:
        projections.append(xim_img.array)
    except:
        pass

In [ ]:
for n in range(len(projections)):
    if not n%100:
        plt.figure()
        plt.imshow(projections[n])
        plt.show()

In [ ]:
def load_projections(filepath, geometry, angles, **kwargs):

    xim_files = os.dir(os.path.join(filepath, "**", "Proj_*.xim"))

    # load images
    xim_files = sorted(
        [file for file in os.listdir(folder) if file.startswith("Proj_") and file.endswith(".xim")]
    )

    return projections, angles, airnorm
